In [55]:
import os
import sys

# Получите путь к родительскому каталогу
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
import mlflow.pytorch
import mlflow
from torchvision.datasets import MNIST
from models import Net
import torch
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [57]:
def train(model, train_loader, optimizer):
    model.train()
    correct = 0  # Счетчик правильных предсказаний
    total = 0    # Общее количество примеров
    for i, (data, target) in enumerate(train_loader):
        print(i, len(train_loader))
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        # Подсчет метрик
        _, predicted = output.max(1)  # Получаем предсказанные классы
        total += target.size(0)       # Обновляем общее количество примеров
        correct += predicted.eq(target).sum().item()  # Обновляем счетчик правильных предсказаний

    accuracy = correct / total  # Вычисляем точность
    return accuracy  # Возвращаем точность

def train_and_log_model(lr):
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Загружаем набор данных MNIST
    dataset1 = MNIST("../data", train=True, download=True, transform=transform)

    # Создаем DataLoader с ограниченным набором данных
    train_loader = DataLoader(dataset1, batch_size=64, shuffle=True)

    print('Dataset собран')
    with mlflow.start_run():
        mlflow.log_param('lr', lr)
        model = Net()
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        print(lr, 'Обучение идет')
        accuracy = train(model, train_loader, optimizer)  # Получаем точность
        mlflow.log_metric('accuracy', accuracy)  # Логируем точность
        run_id = mlflow.active_run()._info.run_id
        model_uri = f"runs:/{run_id}/mnist_app"
        registered_model = mlflow.register_model(model_uri, "Mnist_app")
        client = mlflow.MlflowClient()

        client.set_model_version_tag(
            name="Mnist_app",
            version=registered_model.version,
            key='accuracy',
            value=str(round(accuracy, 3))
        )

In [58]:
def promote_best_model(model_name):
    client = mlflow.MlflowClient()
    best_accuracy = 0
    best_version = None
    for version in client.search_registered_models(f"name='{model_name}'"):
        tmp_accuracy = version.tags.get('accuracy')
        if tmp_accuracy:
            tmp_accuracy = float(tmp_accuracy)
            if tmp_accuracy > best_accuracy:
                best_accuracy = tmp_accuracy
                best_version = version
    if best_version:
        client.transition_model_version_stage(
            name=best_version.name,
            version=best_version,
            stage='Production'
        )

In [59]:
if __name__ == '__main__':
    mlflow.end_run()
    train_and_log_model(0.1)
    train_and_log_model(0.01)
    train_and_log_model(0.001)
    promote_best_model('Mnist_app')

Dataset собран
0.1 Обучение идет
0 938
1 938
2 938
3 938
4 938
5 938
6 938
7 938
8 938
9 938
10 938
11 938
12 938
13 938
14 938
15 938
16 938
17 938
18 938
19 938
20 938
21 938
22 938
23 938
24 938
25 938
26 938
27 938
28 938
29 938
30 938
31 938
32 938
33 938
34 938
35 938
36 938
37 938
38 938
39 938
40 938
41 938
42 938
43 938
44 938
45 938
46 938
47 938
48 938
49 938
50 938
51 938
52 938
53 938
54 938
55 938
56 938
57 938
58 938
59 938
60 938
61 938
62 938
63 938
64 938
65 938
66 938
67 938
68 938
69 938
70 938
71 938
72 938
73 938
74 938
75 938
76 938
77 938
78 938
79 938
80 938
81 938
82 938
83 938
84 938
85 938
86 938
87 938
88 938
89 938
90 938
91 938
92 938
93 938
94 938
95 938
96 938
97 938
98 938
99 938
100 938
101 938
102 938
103 938
104 938
105 938
106 938
107 938
108 938
109 938
110 938
111 938
112 938
113 938
114 938
115 938
116 938
117 938
118 938
119 938
120 938
121 938
122 938
123 938
124 938
125 938
126 938
127 938
128 938
129 938
130 938
131 938
132 938
133 938
134 9

Successfully registered model 'Mnist_app'.
2025/03/10 17:12:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Mnist_app, version 1


937 938
🏃 View run worried-skunk-5 at: http://127.0.0.1:5000/#/experiments/0/runs/1c124d6254244d098bfa4a5846d240b3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
Dataset собран


Created version '1' of model 'Mnist_app'.


0.01 Обучение идет
0 938
1 938
2 938
3 938
4 938
5 938
6 938
7 938
8 938
9 938
10 938
11 938
12 938
13 938
14 938
15 938
16 938
17 938
18 938
19 938
20 938
21 938
22 938
23 938
24 938
25 938
26 938
27 938
28 938
29 938
30 938
31 938
32 938
33 938
34 938
35 938
36 938
37 938
38 938
39 938
40 938
41 938
42 938
43 938
44 938
45 938
46 938
47 938
48 938
49 938
50 938
51 938
52 938
53 938
54 938
55 938
56 938
57 938
58 938
59 938
60 938
61 938
62 938
63 938
64 938
65 938
66 938
67 938
68 938
69 938
70 938
71 938
72 938
73 938
74 938
75 938
76 938
77 938
78 938
79 938
80 938
81 938
82 938
83 938
84 938
85 938
86 938
87 938
88 938
89 938
90 938
91 938
92 938
93 938
94 938
95 938
96 938
97 938
98 938
99 938
100 938
101 938
102 938
103 938
104 938
105 938
106 938
107 938
108 938
109 938
110 938
111 938
112 938
113 938
114 938
115 938
116 938
117 938
118 938
119 938
120 938
121 938
122 938
123 938
124 938
125 938
126 938
127 938
128 938
129 938
130 938
131 938
132 938
133 938
134 938
135 938
136

...

Created version '2' of model 'Mnist_app'.


Dataset собран
0.001 Обучение идет
0 938
1 938
2 938
3 938
4 938
5 938
6 938
7 938
8 938
9 938
10 938
11 938
12 938
13 938
14 938
15 938
16 938
17 938
18 938
19 938
20 938
21 938
22 938
23 938
24 938
25 938
26 938
27 938
28 938
29 938
30 938
31 938
32 938
33 938
34 938
35 938
36 938
37 938
38 938
39 938
40 938
41 938
42 938
43 938
44 938
45 938
46 938
47 938
48 938
49 938
50 938
51 938
52 938
53 938
54 938
55 938
56 938
57 938
58 938
59 938
60 938
61 938
62 938
63 938
64 938
65 938
66 938
67 938
68 938
69 938
70 938
71 938
72 938
73 938
74 938
75 938
76 938
77 938
78 938
79 938
80 938
81 938
82 938
83 938
84 938
85 938
86 938
87 938
88 938
89 938
90 938
91 938
92 938
93 938
94 938
95 938
96 938
97 938
98 938
99 938
100 938
101 938
102 938
103 938
104 938
105 938
106 938
107 938
108 938
109 938
110 938
111 938
112 938
113 938
114 938
115 938
116 938
117 938
118 938
119 938
120 938
121 938
122 938
123 938
124 938
125 938
126 938
127 938
128 938
129 938
130 938
131 938
132 938
133 938
134

Registered model 'Mnist_app' already exists. Creating a new version of this model...
2025/03/10 17:13:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Mnist_app, version 3


935 938
936 938
937 938
🏃 View run clean-bug-110 at: http://127.0.0.1:5000/#/experiments/0/runs/b3409f2a49bf4fa09528722645d7632e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '3' of model 'Mnist_app'.


In [53]:
import requests
response = requests.get("http://127.0.0.1:5000/api/2.0/mlflow/runs/get")
print(response.status_code)
print(response.json())

400
{'error_code': 'INVALID_PARAMETER_VALUE', 'message': "Missing value for required parameter 'run_id'. See the API docs for more information about request parameters."}
